## 모델 저장과 복원
---
 - 방법1) 모델 전체 저장(모델 구조+가중치+변수)
   - save_model()
   - 복원시 추가 작업 필요 없이 모델 파일 로딩해서 사용 가능

 - 방법2) 가중치만 저장
   - save_weights()
   - 복원시 모델 구조(Architechture) 생성 후 가중치 적용

 - 방법3) 모델 전체 또는 자동 저장
   - fit()에서 ModelCheckPoint Event에 대한 callbacks 등록
   - save_best_only=True: 모니터링 기준에 따라서 좋은 성능의 모델만 저장
   - save_weight_only=True: 가중치만 저장
 
 - 파일 또는 폴더
   - 파일 확장자가 없으면 폴더로 저장
   - 파일 확장자
     - h5 / hdf5: HDF5 포맷으로 모델 또는 가중치 저장
     - ckpf: 체크포인트파일형태로 저장
     - pd: 모델 저장

In [51]:
from sklearn.datasets import load_iris
from keras import Sequential
from keras.layers import Dense
from keras.utils import set_random_seed, plot_model

import numpy as np

## [1] 데이터 준비 및 로딩

In [59]:
iris=load_iris()
x_train=iris.data
y_train=iris.target
x_train.shape, y_train.shape

((150, 4), (150,))

In [60]:
iris.target_names, iris.feature_names

(array(['setosa', 'versicolor', 'virginica'], dtype='<U10'),
 ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'])

In [61]:
# Feature -> 꽃잎의 길이, 너비
x_train=x_train[:,2:]
x_train.shape

(150, 2)

In [62]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)
x_train_scaled=scaler.transform(x_train)

In [63]:
# target: 문자 -> 수치화
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

lencoder=LabelEncoder()
y_train=lencoder.fit(y_train).transform(y_train)

In [64]:
onencoder=OneHotEncoder()
y_train=onencoder.fit(y_train.reshape(-1,1)).transform(y_train.reshape(-1,1))

In [65]:
y_train=y_train.toarray()

In [66]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x_train,y_train)
x_train, x_val, y_train, y_val= train_test_split(x_train,y_train)

In [67]:
x_train.shape, x_test.shape, x_val.shape

((84, 2), (38, 2), (28, 2))

### [3] 모델 구성 및 생성

In [82]:
irisModel=Sequential(name='irisModel')

In [83]:
irisModel.add(Dense(150, activation='relu', input_shape=(2,)))
irisModel.add(Dense(150, activation='relu'))
irisModel.add(Dense(3, activation='softmax'))

In [84]:
irisModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [85]:
irisModel.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
3/3 [==============================] - 1s 172ms/step - loss: 1.2018 - accuracy: 0.2976 - val_loss: 1.0338 - val_accuracy: 0.2632
Epoch 2/100
3/3 [==============================] - 0s 24ms/step - loss: 1.0274 - accuracy: 0.3452 - val_loss: 1.0916 - val_accuracy: 0.2632
Epoch 3/100
3/3 [==============================] - 0s 26ms/step - loss: 1.0296 - accuracy: 0.3452 - val_loss: 1.1103 - val_accuracy: 0.2632
Epoch 4/100
3/3 [==============================] - 0s 30ms/step - loss: 1.0164 - accuracy: 0.3452 - val_loss: 1.0430 - val_accuracy: 0.2632
Epoch 5/100
3/3 [==============================] - 0s 32ms/step - loss: 0.9813 - accuracy: 0.3452 - val_loss: 0.9906 - val_accuracy: 0.2632
Epoch 6/100
3/3 [==============================] - 0s 32ms/step - loss: 0.9665 - accuracy: 0.3452 - val_loss: 0.9587 - val_accuracy: 0.2632
Epoch 7/100
3/3 [==============================] - 0s 32ms/step - loss: 0.9568 - accuracy: 0.3452 - val_loss: 0.9473 - val_accuracy: 0.2632
Epoch 8/100
3/3 [==

## [5] 모델 저장

In [93]:
lossv, accv=irisModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.2577 - accuracy: 0.9211


In [94]:
accv

0.9210526347160339

In [97]:
M_FILE='irisModel.h5'
W_FILE='irisWeights.h5'

In [98]:
from keras.models import save_model
if accv >=0.9:
    save_model(irisModel, M_FILE)
    irisModel.save_weights(W_FILE)

## [6] 복원
---
 - 모델 또는 가중치 복원
 - load_model(모델 파일 또는 모델 폴더명)
 - Sequential.load_weights(가중치 파일 또는 폴더 명)


### [6-1] 모델 복원

In [99]:
from keras.models import load_model

In [100]:
reModel=load_model(M_FILE)
reModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.2577 - accuracy: 0.9211


[0.2577494978904724, 0.9210526347160339]

### [6-2] 가중치복원
 - 모델 구조 생성
 - 가중치 로딩

In [101]:
# 모델 구조 구성
newModel=Sequential(name='newirisModel')
newModel.add(Dense(150, activation='relu', input_shape=(2,)))
newModel.add(Dense(150, activation='relu'))
newModel.add(Dense(3, activation='softmax'))

In [105]:
# 모델 생성
newModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [106]:
# 가중치 로딩
newModel.load_weights(W_FILE)

In [107]:
newModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.2577 - accuracy: 0.9211


[0.2577494978904724, 0.9210526347160339]